In [1]:
%matplotlib inline
import warnings; warnings.simplefilter('ignore')

import fnmatch
import os
import pandas as pd
import geopandas as gpd

from datacube_stats.statistics import GeoMedian
from datacube.helpers import ga_pq_fuser
from datacube.storage import masking
from datacube.helpers import write_geotiff
import xarray as xr

#get the DEA version of the plotting functions
import sys
sys.path.append(os.path.abspath('/g/data/r78/DPIPWE_lm/dea-notebooks/10_Scripts'))
import DEAPlotting
import DEADataHandling


import datacube
dc = datacube.Datacube(app='stats_2nd_testing')

In [2]:
#x, y = (1385000.0, 1390000.0), (-4570000.0, -4575000.0)
sensors = ['ls8'] #take or remove as needed
deriv = 'nbart'
#product = 'nbart' #
time = ('2016-10-01', '2016-12-31')
resolution = (-50,50)
bands = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2']
#epoch = ('2016', '2017') # time query for datacube function can be just years

In [3]:
query = {'x': (1300000.0, 1400000.0),
         'y': (-4700000.0, -4800000.0),
         'time': time,
         'resolution': resolution,
         'crs': 'EPSG:3577'}

In [4]:
dsma = DEADataHandling.load_clearlandsat(dc=dc, query=query,
                                          #product=product,
                                          masked_prop=0,
                                          sensors = sensors,
                                          bands_of_interest = bands,
                                          mask_pixel_quality=True,
                                          ls7_slc_off=True)

Loading ls8 pixel quality
    Loading 12 filtered ls8 timesteps
Combining and sorting ls8 data
    Replacing invalid -999 values with NaN (data will be coerced to float64)


In [5]:
dsma

<xarray.Dataset>
Dimensions:    (time: 12, x: 2000, y: 2000)
Coordinates:
  * y          (y) float64 -4.7e+06 -4.7e+06 -4.7e+06 ... -4.8e+06 -4.8e+06
  * x          (x) float64 1.3e+06 1.3e+06 1.3e+06 ... 1.4e+06 1.4e+06 1.4e+06
  * time       (time) datetime64[ns] 2016-10-03T23:52:33 ... 2016-12-31T23:46:20
Data variables:
    red        (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    green      (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    blue       (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    nir        (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    swir1      (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    swir2      (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    data_perc  (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    crs:      EPSG:3577

In [37]:
from model import SMAD, TernaryMAD

In [29]:
SMAD(dsma)

In [38]:
dsma_smad = TernaryMAD().compute(dsma)

In [39]:
dsma_smad

<xarray.Dataset>
Dimensions:  (x: 2000, y: 2000)
Dimensions without coordinates: x, y
Data variables:
    sdev     (y, x) float32 0.0012462151 0.004304504 ... 0.0007072464
    edev     (y, x) float32 0.017819311 0.050129905 ... 0.028593507 0.0366869
    bcdev    (y, x) float32 0.033000797 0.07529552 ... 0.13990852 0.17620553

In [46]:
dsma_smad.set_coords(['x', 'y'])

ValueError: One or more of the specified variables cannot be found in this dataset

In [22]:
dsma_smad.attrs['crs'] = 'EPSG:3577'
dsma_smad.attrs['affine'] = dsma.affine

In [25]:
datacube.storage.storage.write_dataset_to_netcdf(dsma_smad, 'ls8_smad_test.nc')
#datacube.helpers.write_geotiff(filename='ls8_smad_test.tif', dataset=dsma_smad)
#DEADataHandling.dataset_to_geotiff('dsma_smad_netcdf_test.nc', dsma_smad)

AttributeError: 'str' object has no attribute 'geographic'